# 🤖 Lab 4 – AI Meeting Assistant (Tool‑Enabled Agentic AI)

Welcome to **Lab 4**, where you'll move beyond simple prompts and build your **first real Agent** — a system that can **think, plan, and act**.

---
## 🧩 Understanding Agents vs LLMs (Before You Code)
---

## 🧠 Step 1: What Is an LLM?
A **Large Language Model (LLM)** takes a prompt and returns text — it doesn’t remember or act.

Example:

In [ ]:
# Simple LLM call
response = client.responses.create(
    model="gpt-4o-mini",
    input="Summarize this meeting transcript.",
    temperature=0.7
)
print(response.output_text)  # just text output

🟢 **LLM = reasoning only**, it **cannot perform actions** like sending an email or extracting files.

## ⚙️ Step 2: Why Plain LLMs Aren’t Enough
In real-world apps, we need our AI to *do things*:
- Summarize a meeting
- Extract action items
- Send an email or push alert

➡️ Plain LLMs can’t perform these actions — they only generate text.

## 🤖 Step 3: What Is an Agent?
An **Agent** = 🧠 **LLM (brain)** + 🧰 **Tools (hands)** + 🧭 **Logic (manager)**.

| Component | Role |
|------------|------|
| LLM | Thinks & reasons |
| Tools | Perform actions (email, summarize, notify) |
| Resrouces | Knowledge base (database, context, customized data - pdf, excel, etc) |
| Agent | Plans and orchestrates everything |

> The Agent decides *which tool to use, when, and how*.

## 🧩 Step 4: Agent Workflow
```
User → Agent → LLM → (decides) → Tool 1 / Tool 2 / Tool 3 → Result
```
In our **Meeting Assistant**:
1️⃣ User asks → “Summarize & notify team.”  
2️⃣ Agent → runs `transcript_summary_tool`  
3️⃣ Agent → runs `action_item_extractor_tool`  
4️⃣ Agent → calls `send_email_tool` + `send_push_notification_tool`  

### 🧠 Extended Agent Architecture (Including Resources & Memory)

Agents can also work with **additional optional components** like **resources**, **memory**, and **external APIs**.

```
User → Agent → LLM → (decides) → Tools / Resources / Memory / External APIs → Result
```

#### 🧩 Core Components
| Component | Description | Example |
|------------|--------------|----------|
| 🧠 **LLM** | The reasoning engine — interprets and generates text. | GPT‑4o, Claude, Gemini, Ollama |
| 🧭 **Agent** | The orchestrator — decides what to do next and manages context. | Meeting Assistant, Sales Agent |
| 🧰 **Tools** | Functions or APIs the Agent can call to act. | Email sender, Database fetcher, Web search tool |

#### 🪄 Optional Components
| Component | Role | Example |
|------------|------|----------|
| 📚 **Resources** | Static or dynamic data sources used by the Agent for grounding. | PDFs, RAG vector DB, JSON files, APIs |
| 🧠 **Memory** | Stores conversational or long‑term context for continuity. | Remember previous meeting summary |
| 🌐 **External APIs** | Real‑world integrations for actions. | SendGrid, Pushover, Google Calendar |
| 🧩 **Reasoning Logic** | Internal control flow (looping, reflection, evaluation). | Plan → Act → Reflect |
| 🎛️ **Orchestrator / Runner** | Manages tool execution and handles flow tracing. | `Runner.run(agent, task)` |

#### 🔁 Putting It All Together
```
            ┌───────────────────────────────┐
            │           USER INPUT          │
            └──────────────┬────────────────┘
                           ↓
                    ┌──────────────┐
                    │   AGENT      │
                    │ (Planner +   │
                    │  Controller) │
                    └──────┬───────┘
                           ↓
               ┌───────────────────────┐
               │        LLM            │
               │ (Thinks / Decides)    │
               └─────────┬─────────────┘
                         ↓
     ┌─────────────────────────────────────────────┐
     │ TOOLS │ RESOURCES │ MEMORY │ EXTERNAL APIs │
     │ (Actions) (Data)   (Context) (Integrations) │
     └─────────────────────────────────────────────┘
                           ↓
                     📤 Final Output
```

**Summary:** An Agent is a complete system that uses an **LLM for reasoning**, **tools for actions**, **resources for knowledge**, and **memory for context** — all working together to achieve intelligent outcomes.

## 🪄 Step 5: One-Line Definition
> “An **Agent** is a **project manager for AI** — it tells the LLM *what to do* and uses tools to *get it done*.”

## 💻 Step 6: Connecting Code to Concept

In [ ]:
from agents import Agent

meeting_agent = Agent(
    name="Meeting Manager",
    model="gpt-4o-mini",
    instructions="You can summarize, extract actions, and notify team members.", # system prompt
    tools=[transcript_summary_tool, action_item_extractor_tool, send_email_tool]
)
print(meeting_agent)  # shows registered tools and model

## 🏁 Step 7: Real-World Analogy
Think of **ChatGPT** as your *brain* — it can reason but can’t send emails.  
The **Agent** is ChatGPT + hands + eyes + a planner — it can **act** and **delegate tasks**.  

---

## 🪄 Step 8: Install Dependencies

In [1]:
!uv pip install openai gradio sendgrid requests

Using Python 3.12.11 environment at: E:\agentic-dpoint\.venv
Resolved 65 packages in 1.06s
Uninstalled 2 packages in 11ms
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 1 package in 37ms
 - pydantic-core==2.41.4
 ~ pydantic-core==2.33.2


## ⚙️ Step 9: Setup & Imports

In [1]:
import os
from openai import OpenAI
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail
import requests

# Initialize API clients
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
SENDGRID_API_KEY = os.getenv('SENDGRID_API_KEY')
PUSHOVER_USER_KEY = os.getenv('PUSHOVER_USER_KEY')
PUSHOVER_API_TOKEN = os.getenv('PUSHOVER_API_TOKEN')

## 🧰 Step 10: Define Tool Functions - manual/explicit calling

In [2]:
def transcript_summary_tool(transcript: str) -> str:
    '''Summarize the meeting transcript.''' 
    prompt = f'Summarize the following meeting transcript into concise bullet points:\n{transcript}'
    res = client.responses.create(model='gpt-4o-mini', input=prompt)
    return res.output_text

def action_item_extractor_tool(transcript: str) -> str:
    '''Extract key action items from the meeting.''' 
    prompt = f'List all actionable items discussed in the meeting with clear owners if mentioned:\n{transcript}'
    res = client.responses.create(model='gpt-4o-mini', input=prompt)
    return res.output_text

def send_email_tool(to_email: str, subject: str, content: str) -> str:
    '''Send meeting summary via SendGrid email.''' 
    try:
        message = Mail(from_email='tejapolisettys47@gmail.com', to_emails=to_email, subject=subject, plain_text_content=content)
        sg = SendGridAPIClient(SENDGRID_API_KEY)
        sg.send(message)
        return f'✅ Email sent successfully to {to_email}'
    except Exception as e:
        return f'❌ Email failed: {e}'

def send_push_notification_tool(title: str, message: str) -> str:
    '''Send mobile push notification using Pushover.''' 
    try:
        payload = {'token': PUSHOVER_API_TOKEN, 'user': PUSHOVER_USER_KEY, 'title': title, 'message': message}
        response = requests.post('https://api.pushover.net/1/messages.json', data=payload)
        if response.status_code == 200:
            return '📲 Notification sent successfully!'
        else:
            return f'❌ Failed to send notification: {response.text}'
    except Exception as e:
        return f'❌ Pushover error: {e}'

In [3]:
# validate push tool
send_push_notification_tool("hi","hello2")


'📲 Notification sent successfully!'

In [4]:
# validate email tool
send_email_tool('dpointyt@gmail.com','test','testcontent')

'✅ Email sent successfully to dpointyt@gmail.com'

## 🧩 Step 11:  Add Gradio UI

In [5]:
import gradio as gr

def run_meeting_assistant(transcript, email):
    summary = transcript_summary_tool(transcript)
    actions = action_item_extractor_tool(transcript)
    email_content = f'Meeting Summary:\n{summary}\n\nAction Items:\n{actions}'
    email_status = send_email_tool(email, 'AI Meeting Summary', email_content)
    send_push_notification_tool("summary",email_status)
    return f'{email_status}\n\n---\n{email_content}'

with gr.Blocks() as demo:
    gr.Markdown('## 🤖 AI Meeting Assistant – Summarize, Extract & Notify')
    transcript_input = gr.Textbox(label='Paste Meeting Transcript', lines=8)
    email_input = gr.Textbox(label='Send Summary To (Email)')
    output_box = gr.Textbox(label='Output', lines=15)
    run_btn = gr.Button('Run Assistant')
    run_btn.click(run_meeting_assistant, inputs=[transcript_input, email_input], outputs=output_box)

demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## 🧰 Step 12: Define Tool Functions - for the agent

In [7]:
from agents import Agent, Runner, trace,function_tool


@function_tool
def transcript_summary_tool(transcript: str) -> str:
    '''Summarize the meeting transcript.''' 
    prompt = f'Summarize the following meeting transcript into concise bullet points:\n{transcript}'
    res = client.responses.create(model='gpt-4o-mini', input=prompt)
    return res.output_text

@function_tool
def action_item_extractor_tool(transcript: str) -> str:
    '''Extract key action items from the meeting.''' 
    prompt = f'List all actionable items discussed in the meeting with clear owners if mentioned:\n{transcript}'
    res = client.responses.create(model='gpt-4o-mini', input=prompt)
    return res.output_text

@function_tool
def send_email_tool(to_email: str, subject: str, content: str) -> str:
    '''Send meeting summary via SendGrid email.''' 
    try:
        message = Mail(from_email='tejapolisettys47@gmail.com', to_emails=to_email, subject=subject, plain_text_content=content)
        sg = SendGridAPIClient(SENDGRID_API_KEY)
        sg.send(message)
        return f'✅ Email sent successfully to {to_email}'
    except Exception as e:
        return f'❌ Email failed: {e}'

@function_tool
def send_push_notification_tool(title: str, message: str) -> str:
    '''Send mobile push notification using Pushover.''' 
    try:
        payload = {'token': PUSHOVER_API_TOKEN, 'user': PUSHOVER_USER_KEY, 'title': title, 'message': message}
        response = requests.post('https://api.pushover.net/1/messages.json', data=payload)
        if response.status_code == 200:
            return '📲 Notification sent successfully!'
        else:
            return f'❌ Failed to send notification: {response.text}'
    except Exception as e:
        return f'❌ Pushover error: {e}'

## 🧠 Step 13: Create the Agent and Register Tools

In [9]:


tools = [
    transcript_summary_tool,
    action_item_extractor_tool,
    send_email_tool,
    send_push_notification_tool
]

instructions = '''You are an AI Meeting Assistant.
You have tools to summarize, extract actions, and send notifications.
Always begin by summarizing, then extracting, then notifying.'''

meeting_agent = Agent(name='Meeting Manager', instructions=instructions, model='gpt-4o-mini', tools=tools)
meeting_agent

Agent(name='Meeting Manager', handoff_description=None, tools=[FunctionTool(name='transcript_summary_tool', description='Summarize the meeting transcript.', params_json_schema={'properties': {'transcript': {'title': 'Transcript', 'type': 'string'}}, 'required': ['transcript'], 'title': 'transcript_summary_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001D8F6043880>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='action_item_extractor_tool', description='Extract key action items from the meeting.', params_json_schema={'properties': {'transcript': {'title': 'Transcript', 'type': 'string'}}, 'required': ['transcript'], 'title': 'action_item_extractor_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_t

## 🚀 Step 14: Run the Meeting Assistant

In [10]:
meeting_transcript = """
The DPoint AI engineering and training team met to review progress on the new Agentic AI
training platform and internal client tool development.

Sai Teja opened the meeting by highlighting that the OpenAI SDK-based tool integrations are
working smoothly, but the PDF resource loader still faces latency issues when handling large
files. He mentioned that optimizing the loader should be a top priority for the next sprint.

Ananya presented updates to the Gradio dashboard for student interaction.
The new layout supports custom agent configurations for training sessions, and feedback from
the beta users was encouraging. She suggested connecting the dashboard with client APIs
to automatically record tool usage data.

Poojith shared a proposal to introduce a “Tool Registry” view in the dashboard where users
can explore available tools such as summarization, email automation using SendGrid,
and push notifications via Pushover. The team discussed adding analytics to track which
tools are used most often.

The meeting concluded with Sai Teja suggesting an internal demo recording next week
for DPoint’s marketing and sales team to highlight how different agents and tools
coordinate together in real time.
"""



user_request = f"""
    You are an AI meeting assistant.
    Summarize the following transcript, extract key action items,
    and notify the team via email and push notification.

    --- Meeting Transcript ---
    {meeting_transcript}
    """
    

In [11]:
with trace(workflow_name='Meeting Assistant Flow Demo 1'):
    result = await Runner.run(meeting_agent, user_request)
    print(result.final_output)

### Meeting Summary
The DPoint AI engineering and training team met to review progress on the Agentic AI training platform and internal client tool development.

- **OpenAI SDK Updates:** Successful tool integrations noted, but the PDF resource loader has latency issues with large files; prioritization for optimization in the next sprint.
- **Gradio Dashboard Updates:** Improvements allow custom agent configurations; positive feedback from beta users. Suggested to connect the dashboard with client APIs for automatic tool usage data recording.
- **Tool Registry Proposal:** Proposal to add a "Tool Registry" view to explore available tools and analytics to monitor usage.
- **Next Steps:** An internal demo recording for DPoint’s marketing and sales team will be organized for next week to showcase coordination of agents/tools.

### Action Items
1. **Optimize PDF Resource Loader** - Sai Teja will prioritize optimization for the next sprint.
2. **Connect Gradio Dashboard with Client APIs** - 

## 🧩 Step 15: Add Gradio UI

In [ ]:
import gradio as gr

async def run_meeting_assistant(transcript, email):
    # summary = transcript_summary_tool(transcript)
    # actions = action_item_extractor_tool(transcript)
    # email_content = f'Meeting Summary:\n{summary}\n\nAction Items:\n{actions}'
    # email_status = send_email_tool(email, 'AI Meeting Summary', email_content)
    # return f'{email_status}\n\n---\n{email_content}'
    
    """Gradio handler that delegates to the meeting agent."""
    if not transcript.strip():
        return "⚠️ Please provide a meeting transcript."
    if not email.strip():
        return "⚠️ Please enter a recipient email address."

    # Combine transcript and email into a single user request for the agent
    user_request = f"""
    You are an AI meeting assistant.
    Summarize the following meeting transcript, extract key action items,
    and send a summary email to {email}
    and push the email content to mobile using pushover.
    
    --- Meeting Transcript ---
    {transcript}
    """

    with trace(workflow_name="Gradio Meeting Assistant (Agentic Flow) Demo 2"):
        result = await Runner.run(meeting_agent, user_request)
        return result.final_output

with gr.Blocks() as demo:
    gr.Markdown('## 🤖 AI Meeting Assistant – Summarize, Extract & Notify')
    transcript_input = gr.Textbox(label='Paste Meeting Transcript', lines=8)
    email_input = gr.Textbox(label='Send Summary To (Email)')
    output_box = gr.Textbox(label='Output', lines=15)
    run_btn = gr.Button('Run Assistant')
    run_btn.click(run_meeting_assistant, inputs=[transcript_input, email_input], outputs=output_box)

demo.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


## 🏁 Step 14: Key Takeaways
- You learned the **difference between LLMs and Agents**.
- You built your **first Agent** using multiple tools.
- You integrated **SendGrid and Pushover** for automation.
- You demonstrated **real-world tool orchestration** inside an Agentic AI workflow.